In [6]:
from FPTU_FA24_EEG_Artifacts_Recognition.utils import *
from FPTU_FA24_EEG_Artifacts_Recognition.constants import *
from pathlib import Path

In [8]:
a = read_yaml(Path(r'D:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\config\dataset.yaml'))


2024-11-18 10:51:55,082: INFO: common: yaml file 'D:\Dang_project\FPTU_FA24_EEG_Artifacts_Recognition\config\dataset.yaml' loaded successfully


In [12]:
a.subject.keys()

dict_keys([0, 1, 2])

In [14]:
a.subject[0]

ConfigBox({'name': 'Dang Nguyen Minh', 'position': 3, 'trial': 10})